In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import NuSVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Step 1: Load the dataset
df = pd.read_excel("/content/Dry_Bean_Dataset.xlsx")

# Step 2: Encode the target labels
le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])

# Step 3: Split into features and labels
X = df.drop(columns=['Class'])
y = df['Class']

# Step 4: Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 5: Define SVM parameter space
param_grid = {
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'nu': np.round(np.linspace(0.01, 0.3, 20), 3)
}

# Step 6: Initialize result containers
results = []
convergence_data = []
iterations = 30  # reduce from 100 for faster execution

# Step 7: Repeat 10 times with different train-test splits
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.3, stratify=y, random_state=i)

    best_acc = 0
    best_params = {}
    acc_per_iter = []

    for _ in range(iterations):
        try:
            params = {
                'kernel': np.random.choice(param_grid['kernel']),
                'nu': np.random.choice(param_grid['nu']),
            }
            model = NuSVC(**params)
            model.fit(X_train, y_train)
            preds = model.predict(X_test)
            acc = accuracy_score(y_test, preds)
            acc_per_iter.append(acc)

            if acc > best_acc:
                best_acc = acc
                best_params = params
        except ValueError:
            acc_per_iter.append(0)  # In case of invalid `nu` value

    results.append({
        'Sample': f'S{i+1}',
        'Best Accuracy': round(best_acc * 100, 2),
        'Best Parameters': f"{best_params.get('kernel', '-')}, {round(best_params.get('nu', 0), 2)}"
    })
    convergence_data.append(acc_per_iter)

# Step 8: Save results to Excel
results_df = pd.DataFrame(results)
results_df.to_excel("Optimized_SVM_Results.xlsx", index=False)

# Step 9: Plot the convergence graph for the best performing sample
best_index = np.argmax([r['Best Accuracy'] for r in results])
best_convergence = convergence_data[best_index]

plt.figure(figsize=(8, 5))
plt.plot(best_convergence)
plt.xlabel("Iteration")
plt.ylabel("Accuracy")
plt.title("Fitness (bestAccuracy)")
plt.grid(True)
plt.savefig("convergence_plot.png")
plt.show()
